In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
xcel = "SMOTERegressionCases.xlsx"

# Load each sheet separately
df1 = pd.read_excel(xcel, sheet_name='Seed410_thr0.77_perc0.2')
df2 = pd.read_excel(xcel, sheet_name='Seed527_thr0.77_perc0.2')
df3 = pd.read_excel(xcel, sheet_name='Seed222_thr0.77_perc0.2')
df4 = pd.read_excel(xcel, sheet_name='Seed326_thr0.77_perc0.2')
df5 = pd.read_excel(xcel, sheet_name='Seed832_thr0.77_perc0.2')

# ## dropping column X, as it is unwanted
# df1 = df1.drop(['X'],axis=1)
# df2 = df2.drop(['X'],axis=1)
# df3 = df3.drop(['X'],axis=1)
# df4 = df4.drop(['X'],axis=1)
# df5 = df5.drop(['X'],axis=1)

In [3]:
df1.head()

,id,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,289.0,18.0,55.0,1.0,165.000000,74.500000,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,233.0,10.0,65.0,1.0,166.911029,80.265922,1.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,309.0,6.0,40.0,0.0,166.911029,80.265922,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,377.0,15.0,56.0,0.0,166.911029,80.265922,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,126.0,8.0,82.0,0.0,166.911029,80.265922,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df2.head()

,id,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,207.0,9.0,64.0,1.0,185.000000,90.700000,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,18.0,5.0,87.0,0.0,166.000000,73.300000,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,161.0,2.0,49.0,0.0,166.911029,80.265922,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,106.0,9.0,91.0,1.0,166.911029,80.265922,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,107.0,16.0,78.0,0.0,158.000000,79.200000,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df = pd.read_csv("../Datasets/new.csv")
df = df.drop("Unnamed: 0",axis=1)

df.rename(columns={' Chlorthalidone': 'Chlorthalidone.1'}, inplace=True)

df.columns = df.columns.str.strip()

df

,id,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,1,21,74,1,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,5,61,0,154.900000,73.600000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,7,58,0,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,9,94,1,182.800000,66.200000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,9,91,1,166.911029,80.265922,1.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,515,2,56,0,165.000000,72.600000,0.0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
502,516,13,74,0,166.911029,55.000000,0.0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
503,517,19,81,1,158.000000,65.000000,0.0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,519,9,74,1,166.911029,81.700000,0.0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
# Concatenate df with df1, df with df2, and df with df3 separately
df1 = pd.concat([df, df1], ignore_index=True)
df2 = pd.concat([df, df2], ignore_index=True)
df3 = pd.concat([df, df3], ignore_index=True)
df4 = pd.concat([df, df4], ignore_index=True)
df5 = pd.concat([df, df5], ignore_index=True)

In [7]:
# Randomize the rows by shuffling the index
df1 = df1.sample(frac=1).reset_index(drop=True)
df2 = df2.sample(frac=1).reset_index(drop=True)
df3 = df3.sample(frac=1).reset_index(drop=True)
df4 = df4.sample(frac=1).reset_index(drop=True)
df5 = df5.sample(frac=1).reset_index(drop=True)

In [8]:
df2

,id,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,387.000000,25.0,69.000000,1.000000,166.911029,67.000000,0.0,0.0,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,230.000000,3.0,34.000000,1.000000,166.911029,70.000000,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,291.269669,38.5,72.288388,0.000000,152.573033,76.023576,0.0,0.0,0.288388,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,319.261048,37.0,76.522096,0.000000,157.500000,67.586333,0.0,0.0,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,45.000000,13.0,56.000000,1.000000,179.000000,80.000000,1.0,1.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,161.000000,2.0,49.000000,0.000000,166.911029,80.265922,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1020,221.415246,32.5,73.116402,0.662355,165.033902,66.306467,0.0,0.0,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1021,440.000000,2.0,60.000000,0.000000,166.911029,80.265922,0.0,0.0,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1022,228.000000,9.0,72.000000,1.000000,180.300000,90.000000,0.0,0.0,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Scaling Age, Height and Weight

In [9]:
from sklearn.preprocessing import StandardScaler

columns_to_standardize = ['age', 'height', 'weight']

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the selected columns using the scaler
df1[columns_to_standardize] = scaler.fit_transform(df1[columns_to_standardize])
df2[columns_to_standardize] = scaler.fit_transform(df2[columns_to_standardize])
df3[columns_to_standardize] = scaler.fit_transform(df3[columns_to_standardize])
df4[columns_to_standardize] = scaler.fit_transform(df4[columns_to_standardize])
df5[columns_to_standardize] = scaler.fit_transform(df5[columns_to_standardize])

### Dropping unwanted columns

In [10]:
df1 = df1.drop(['id'],axis=1)
df2 = df2.drop(['id'],axis=1)
df3 = df3.drop(['id'],axis=1)
df4 = df4.drop(['id'],axis=1)
df5 = df5.drop(['id'],axis=1)

### Applying 0.5 or more to be ceil and lesser to be floor

In [11]:
# Function to apply ceil/floor logic
def ceil_floor(value):
    if value == 0.0:
        return value  # Leave 0.0 unchanged
    elif value >= 0.5:
        return np.ceil(value)  # Apply ceiling for values >= 0.5
    else:
        return np.floor(value)  # Apply floor for values < 0.5

# Step 1: Round to two decimal points
df1 = df1.round(2)
df2 = df2.round(2)
df3 = df3.round(2)
df4 = df4.round(2)
df5 = df1.round(2)

# Specify columns to exclude from rounding and applying ceil/floor logic
exclude_cols = ['age', 'height', 'weight']

# Step 1: Round to two decimal points for all columns except the excluded ones
cols_to_modify = df1.columns.difference(exclude_cols)

# Apply the function to the entire DataFrame
df1[cols_to_modify] = df1[cols_to_modify].applymap(ceil_floor)
df2[cols_to_modify] = df2[cols_to_modify].applymap(ceil_floor)
df3[cols_to_modify] = df3[cols_to_modify].applymap(ceil_floor)
df4[cols_to_modify] = df4[cols_to_modify].applymap(ceil_floor)
df5[cols_to_modify] = df5[cols_to_modify].applymap(ceil_floor)

In [12]:
df1

,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,48.0,1.49,0.0,-0.14,-0.79,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,36.0,-0.02,1.0,1.01,0.56,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.0,-0.31,0.0,-1.78,0.11,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,51.0,0.23,1.0,-0.33,-0.57,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,52.0,-1.45,0.0,-1.47,-0.46,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,62.0,1.51,1.0,-0.01,-0.96,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1020,15.0,1.73,1.0,0.81,-2.16,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1021,15.0,-0.31,1.0,0.06,-0.57,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1022,31.0,-0.59,1.0,-0.61,-0.11,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def test_experiment_1(df):
    # Assuming 'df' has both features (X) and target (y) variables
    X = df.drop('hospital_length_of_stay', axis=1)  # Replace with actual column name
    y = df['hospital_length_of_stay']               # Replace with actual column name

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the models with specified alpha values
    models = {
        'Lasso alpha 0.1': Lasso(alpha=0.1),
        'Lasso alpha 10': Lasso(alpha=10),
        'Ridge alpha 0.1': Ridge(alpha=0.1),
        'Ridge alpha 10': Ridge(alpha=10),
        'RandomForest': RandomForestRegressor(),
        'MLP': MLPRegressor(max_iter=500),  # Increase max_iter if necessary
        'Logistic Regression': LogisticRegression()  # Add logistic regression
    }
    
    # Dictionary to store the results
    results = {}

    # Loop over the models
    for name, model in models.items():
        # Fit the model on the training data
        model.fit(X_train, y_train)
        
        # Make predictions on the training data
        y_train_pred = model.predict(X_train)
        # Make predictions on the test data
        y_test_pred = model.predict(X_test)
        
        # Calculate MSE for training and test data
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
        
        # Store the results
        results[name] = {
            'Train MSE': train_mse,
            'Test MSE': test_mse
        }

    return results

# Run the test_experiment function on each dataset
results_df1 = test_experiment_1(df1)
results_df2 = test_experiment_1(df2)
results_df3 = test_experiment_1(df3)
results_df4 = test_experiment_1(df4)
results_df5 = test_experiment_1(df5)

# Create a dictionary to aggregate the results
aggregated_results = {}

# Add results from each dataset to the aggregated dictionary
for i, results in enumerate([results_df1, results_df2, results_df3, results_df4, results_df5], start=1):
    for model_name, metrics in results.items():
        if model_name not in aggregated_results:
            aggregated_results[model_name] = {
                'Train MSE': [],
                'Test MSE': []
            }
        aggregated_results[model_name]['Train MSE'].append(metrics['Train MSE'])
        aggregated_results[model_name]['Test MSE'].append(metrics['Test MSE'])

# Calculate mean and standard deviation for each model
final_results = {}

for model_name, mse_values in aggregated_results.items():
    avg_train_mse = round(sum(mse_values['Train MSE']) / len(mse_values['Train MSE']), 2)
    std_train_mse = round((sum((x - avg_train_mse) ** 2 for x in mse_values['Train MSE']) / len(mse_values['Train MSE'])) ** 0.5, 2)
    avg_test_mse = round(sum(mse_values['Test MSE']) / len(mse_values['Test MSE']), 2)
    std_test_mse = round((sum((x - avg_test_mse) ** 2 for x in mse_values['Test MSE']) / len(mse_values['Test MSE'])) ** 0.5, 2)
    
    # Combine mean and std into a single string
    final_results[model_name] = {
        'Train MSE (mean ± std)': f"{avg_train_mse} ± {std_train_mse}",
        'Test MSE (mean ± std)': f"{avg_test_mse} ± {std_test_mse}"
    }

# Convert final results to a DataFrame
results_df = pd.DataFrame(final_results).T  # Transpose for better readability

# Display the results
print(results_df)

/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  war

                    Train MSE (mean ± std) Test MSE (mean ± std)
Lasso alpha 0.1              142.84 ± 5.14        168.95 ± 15.63
Lasso alpha 10                274.27 ± 4.3         259.66 ± 9.64
Ridge alpha 0.1               26.12 ± 1.38        166.47 ± 12.37
Ridge alpha 10                90.82 ± 3.21        124.57 ± 14.01
RandomForest                   9.01 ± 0.67         74.23 ± 14.13
MLP                           16.98 ± 1.09         116.07 ± 5.76
Logistic Regression           29.66 ± 1.81         81.46 ± 17.05


In [14]:
results_df.to_csv('results_with_SMOGN.csv')

In [15]:
results_df

,Train MSE (mean ± std),Test MSE (mean ± std)
Lasso alpha 0.1,142.84 ± 5.14,168.95 ± 15.63
Lasso alpha 10,274.27 ± 4.3,259.66 ± 9.64
Ridge alpha 0.1,26.12 ± 1.38,166.47 ± 12.37
Ridge alpha 10,90.82 ± 3.21,124.57 ± 14.01
RandomForest,9.01 ± 0.67,74.23 ± 14.13
MLP,16.98 ± 1.09,116.07 ± 5.76
Logistic Regression,29.66 ± 1.81,81.46 ± 17.05


In [16]:
import pandas as pd
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def test_experiment_2(df):
    # Initialize models with specified alpha values
    models = {
        'Lasso alpha 0.1': Lasso(alpha=0.1),
        'Lasso alpha 10': Lasso(alpha=10),
        'Ridge alpha 0.1': Ridge(alpha=0.1),
        'Ridge alpha 10': Ridge(alpha=10),
        'RandomForest': RandomForestRegressor(),
        'MLP': MLPRegressor(max_iter=500),  # Increase max_iter if necessary
        'Logistic Regression': LogisticRegression(max_iter=500)  # Add Logistic Regression
    }

    # Dictionary to store aggregated MSE results
    aggregated_results = {}

    # Loop to create 5 different subsets using different random states
    for random_state in range(5):
        # Perform the train-test split with varying random states
        X_train, X_test, y_train, y_test = train_test_split(
            df.drop('hospital_length_of_stay', axis=1),  # Drop the target column
            df['hospital_length_of_stay'],               # Target column
            test_size=0.2,                               # 20% for testing
            random_state=random_state                    # Use different random states
        )
        
        # Loop through each model
        for model_name, model in models.items():
            # Fit the model
            model.fit(X_train, y_train)
            
            # Predict on training and test data
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)
            
            # Calculate MSE
            train_mse = mean_squared_error(y_train, y_train_pred)
            test_mse = mean_squared_error(y_test, y_test_pred)
            
            # Store results in aggregated_results dictionary
            if model_name not in aggregated_results:
                aggregated_results[model_name] = {
                    'Train MSE': [],
                    'Test MSE': []
                }
            aggregated_results[model_name]['Train MSE'].append(train_mse)
            aggregated_results[model_name]['Test MSE'].append(test_mse)

    # Calculate mean and std for each model's Train MSE and Test MSE
    final_results = {}

    for model_name, mse_values in aggregated_results.items():
        # Compute average and standard deviation for Train and Test MSE
        avg_train_mse = round(sum(mse_values['Train MSE']) / len(mse_values['Train MSE']), 2)
        std_train_mse = round((sum((x - avg_train_mse) ** 2 for x in mse_values['Train MSE']) / len(mse_values['Train MSE'])) ** 0.5, 2)
        avg_test_mse = round(sum(mse_values['Test MSE']) / len(mse_values['Test MSE']), 2)
        std_test_mse = round((sum((x - avg_test_mse) ** 2 for x in mse_values['Test MSE']) / len(mse_values['Test MSE'])) ** 0.5, 2)
        
        # Combine mean and std into a single string
        final_results[model_name] = {
            'Train MSE (mean ± std)': f"{avg_train_mse} ± {std_train_mse}",
            'Test MSE (mean ± std)': f"{avg_test_mse} ± {std_test_mse}"
        }

    # Convert final results to a DataFrame
    results_df = pd.DataFrame(final_results).T  # Transpose for better readability

    # Return the results
    return results_df

# Example usage
results_df = test_experiment_2(df)
results_df

/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

,Train MSE (mean ± std),Test MSE (mean ± std)
Lasso alpha 0.1,104.39 ± 4.52,128.8 ± 26.56
Lasso alpha 10,140.32 ± 6.64,119.46 ± 26.15
Ridge alpha 0.1,10.97 ± 1.85,260.86 ± 25.58
Ridge alpha 10,76.86 ± 4.8,135.97 ± 24.62
RandomForest,18.64 ± 0.86,114.17 ± 19.08
MLP,108.59 ± 8.75,128.06 ± 25.16
Logistic Regression,205.64 ± 14.02,273.61 ± 40.32


### Hyper parameters for the models

In [17]:
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# Initialize the models with specified alpha values
models = {
    'Lasso alpha 0.1': Lasso(alpha=0.1),
    'Lasso alpha 10': Lasso(alpha=10),
    'Ridge alpha 0.1': Ridge(alpha=0.1),
    'Ridge alpha 10': Ridge(alpha=10),
    'RandomForest': RandomForestRegressor(),
    'MLP': MLPRegressor(max_iter=500),  # Increase max_iter if necessary
    'Logistic Regression': LogisticRegression()  # Add logistic regression
}

# Print hyperparameters for each model
for model_name, model in models.items():
    print(f"{model_name} hyperparameters:\n{model.get_params()}\n")


Lasso alpha 0.1 hyperparameters:
{'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}

Lasso alpha 10 hyperparameters:
{'alpha': 10, 'copy_X': True, 'fit_intercept': True, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}

Ridge alpha 0.1 hyperparameters:
{'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': None, 'solver': 'auto', 'tol': 0.0001}

Ridge alpha 10 hyperparameters:
{'alpha': 10, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': None, 'solver': 'auto', 'tol': 0.0001}

RandomForest hyperparameters:
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_im